In [214]:
# First, import needed dependencies and give them alias
import pandas as pd
import numpy as np
import os

In [215]:
os.getcwd()

'/Users/huiyingzheng/Desktop/GT Databootcamp/Pandas-challenge'

In [216]:
# Second set the file paths for student and school csv files
schools_csv = os.path.join("Resources", "schools_complete.csv")
students_csv = os.path.join("Resources", "students_complete.csv")

In [220]:
schools_df = pd.read_csv(schools_csv)
students_df = pd.read_csv(students_csv)
students_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39170 entries, 0 to 39169
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Student ID     39170 non-null  int64 
 1   student_name   39170 non-null  object
 2   gender         39170 non-null  object
 3   grade          39170 non-null  object
 4   school_name    39170 non-null  object
 5   reading_score  39170 non-null  int64 
 6   math_score     39170 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 2.1+ MB


In [221]:
schools_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   School ID    15 non-null     int64 
 1   school_name  15 non-null     object
 2   type         15 non-null     object
 3   size         15 non-null     int64 
 4   budget       15 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 728.0+ bytes


In [228]:
# To create total number of schools using school ID
print(schools_df.columns)

n_schools = schools_df['School ID'].unique().shape[0]
print("Total number of schools:" + str(n_schools))

n_students = students_df['Student ID'].count()
print("Total number of students:" + str(n_students))


Index(['School ID', 'school_name', 'type', 'size', 'budget'], dtype='object')
Total number of schools:15
Total number of students:39170


In [231]:
# Calcualte the total budget from the school file
total_budget = schools_df['budget'].sum()
print("Total budget: $" + str(total_budget))

Total budget: $24649428


In [235]:
# Calcualte the average test scores from the student file
avg_math_score = round(students_df['math_score'].sum()/n_students,2)
print("Average math score: " + str(avg_math_score))

avg_read_score = round(students_df['reading_score'].sum()/n_students,2)
print("Average reading score: " + str(avg_read_score))

Average math score: 78.99
Average reading score: 81.88


In [247]:
# Calculate the % of passing math and reading, and overall passing rate
print(students_df.describe())

# In order to calculate the pass/fail percentages, I assumed that score >= 70 is passing
students_df['Math Pass'] = students_df['math_score']>=70
math_pass = round(students_df['Math Pass'].sum()*100/n_students,2)
print("% Pass Math:" + str(math_pass))

# althernatively I can use loc() and len() to calculate the %
read_pass = round(students_df.loc[students_df['reading_score']>=70].shape[0]*100/n_students,2)
print("% Pass Reading: " + str(read_pass))

avg_pass = round((math_pass + read_pass)/2,2)
print("Average % of Passing Math and Reading: "+ str(avg_pass))

         Student ID  reading_score    math_score
count  39170.000000    39170.00000  39170.000000
mean   19584.500000       81.87784     78.985371
std    11307.549359       10.23958     12.309968
min        0.000000       63.00000     55.000000
25%     9792.250000       73.00000     69.000000
50%    19584.500000       82.00000     79.000000
75%    29376.750000       91.00000     89.000000
max    39169.000000       99.00000     99.000000
% Pass Math:74.98
% Pass Reading: 85.81
Average % of Passing Math and Reading: 80.4


In [250]:
# Create the high level snapshot at district level, these steps don't need the merge at first
district_df = pd.DataFrame({
    "Total Schools": [n_schools], 
    "Total Students": [n_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_read_score],
    "% Math Pass": [math_pass],
    "% Reading Pass": [read_pass],
    "Overall Pass Rate": [avg_pass]
    
})

# Format all the columns 

district_df["Total Students"] = district_df["Total Students"].map("{:,}".format)
district_df["Total Budget"] = district_df["Total Budget"].map("${:,}".format)

district_df["% Math Pass"] = district_df["% Math Pass"].map("{:.2f}%".format)
district_df["% Reading Pass"] = district_df["% Reading Pass"].map("{:.2f}%".format)
district_df["Overall Pass Rate"] = district_df["Overall Pass Rate"].map("{:.2f}%".format)


district_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Math Pass,% Reading Pass,Overall Pass Rate
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,80.40%


In [12]:
# Combine the datasets into a single dataset, I merged school back to student files, so every student has school information
school_data_complete = pd.merge(students_df, schools_df, how="left", on=["school_name", "school_name"])
school_data_complete["Math Pass 70"] = school_data_complete.math_score >= 70
school_data_complete["Reading Pass 70"] = school_data_complete.reading_score >= 70
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,Math Pass,School ID,type,size,budget,Math Pass 70,Reading Pass 70
0,0,Paul Bradley,M,9th,Huang High School,66,79,True,0,District,2917,1910635,True,False
1,1,Victor Smith,M,12th,Huang High School,94,61,False,0,District,2917,1910635,False,True
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,False,0,District,2917,1910635,False,True
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,False,0,District,2917,1910635,False,False
4,4,Bonnie Ray,F,9th,Huang High School,97,84,True,0,District,2917,1910635,True,True


In [252]:
# Examine if I need to drop some rows from the data due to missing
print(school_data_complete.count())

# Because there is no missing data, the following step is omitted
# school_data_reduced = school_data_complete.dropna(how="any")
# school_data_reduced.count()

# I would like to use groupby to create a metafile by school 

grouped_school_df = school_data_complete.groupby(['School ID'])
grouped_school_df.count()


Student ID         39170
student_name       39170
gender             39170
grade              39170
school_name        39170
reading_score      39170
math_score         39170
Math Pass          39170
School ID          39170
type               39170
size               39170
budget             39170
Math Pass 70       39170
Reading Pass 70    39170
dtype: int64


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,Math Pass,type,size,budget,Math Pass 70,Reading Pass 70
School ID,,,,,,,,,,,,,
0,2917,2917,2917,2917,2917,2917,2917,2917,2917,2917,2917,2917,2917
1,2949,2949,2949,2949,2949,2949,2949,2949,2949,2949,2949,2949,2949
2,1761,1761,1761,1761,1761,1761,1761,1761,1761,1761,1761,1761,1761
3,4635,4635,4635,4635,4635,4635,4635,4635,4635,4635,4635,4635,4635
4,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468
5,2283,2283,2283,2283,2283,2283,2283,2283,2283,2283,2283,2283,2283
6,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858
7,4976,4976,4976,4976,4976,4976,4976,4976,4976,4976,4976,4976,4976
8,427,427,427,427,427,427,427,427,427,427,427,427,427


In [263]:
# Create an overview table that summarizes key metrics about each school, including:
# School names
school_names = grouped_school_df["school_name"].unique()
# School Type
school_types = grouped_school_df.type.unique()
# Total Students
school_n_students = grouped_school_df["Student ID"].count()
# Total School Budget
school_budgets = grouped_school_df.budget.unique()
# Per Student Budget
avg_budget = school_budgets/school_n_students
# Average Math Score
avg_math = grouped_school_df.math_score.sum()/school_n_students
# Average Reading Score
avg_read = grouped_school_df.reading_score.sum()/school_n_students
# % Passing Math
pass_math_school = round(grouped_school_df["Math Pass 70"].sum()/school_n_students, 4)*100
# % Passing Reading
pass_read_school = round(grouped_school_df["Reading Pass 70"].sum()/school_n_students, 4)*100
# Overall Passing Rate (Average of the above two)
avg_pass_rate = round((pass_math_school + pass_read_school)/2,2)
#Create a dataframe to hold the above results
school_level_df = pd.DataFrame({
    "Schools Names": school_names, 
    "Types": school_types,
    "Total Students": school_n_students,
    "Total School Budget": school_budgets,
    "Per Student Budget": avg_budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,  
    "% Math Pass": pass_math_school,    
    "% Reading Pass": pass_read_school,
    "Overall Pass Rate": avg_pass_rate
})


school_level_df["Total Students"] = school_level_df["Total Students"].map("{:,}".format)
school_level_df["Total School Budget"] = school_level_df["Total School Budget"].astype('float').map("${:,}".format)
school_level_df["Per Student Budget"] = school_level_df["Per Student Budget"].astype('float').map("${:,}".format)
school_level_df["Average Math Score"] = school_level_df["Average Math Score"].map("{:.2f}".format)
school_level_df["Average Reading Score"] = school_level_df["Average Reading Score"].map("{:.2f}".format)
school_level_df["% Math Pass"] = school_level_df["% Math Pass"].map("{:.2f}%".format)
school_level_df["% Reading Pass"] = school_level_df["% Reading Pass"].map("{:.2f}%".format)
school_level_df["Overall Pass Rate"] = school_level_df["Overall Pass Rate"].map("{:.2f}%".format)

school_level_df
# I actually have one question here,how to remove the [] from Types and School Names

,Schools Names,Types,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Math Pass,% Reading Pass,Overall Pass Rate
School ID,,,,,,,,,,
0,[Huang High School],[District],"2,917","$1,910,635.0",$655.0,76.63,81.18,65.68%,81.32%,73.50%
1,[Figueroa High School],[District],"2,949","$1,884,411.0",$639.0,76.71,81.16,65.99%,80.74%,73.37%
2,[Shelton High School],[Charter],"1,761","$1,056,600.0",$600.0,83.36,83.73,93.87%,95.85%,94.86%
3,[Hernandez High School],[District],"4,635","$3,022,020.0",$652.0,77.29,80.93,66.75%,80.86%,73.81%
4,[Griffin High School],[Charter],"1,468","$917,500.0",$625.0,83.35,83.82,93.39%,97.14%,95.26%
5,[Wilson High School],[Charter],"2,283","$1,319,574.0",$578.0,83.27,83.99,93.87%,96.54%,95.21%
6,[Cabrera High School],[Charter],"1,858","$1,081,356.0",$582.0,83.06,83.98,94.13%,97.04%,95.58%
7,[Bailey High School],[District],"4,976","$3,124,928.0",$628.0,77.05,81.03,66.68%,81.93%,74.31%
8,[Holden High School],[Charter],427,"$248,087.0",$581.0,83.80,83.81,92.51%,96.25%,94.38%


In [264]:
# Sort and display the top five schools in overall passing rate
school_level_df = school_level_df.sort_values("Overall Pass Rate", ascending=False)
school_level_df.head(5)

,Schools Names,Types,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Math Pass,% Reading Pass,Overall Pass Rate
School ID,,,,,,,,,,
6,[Cabrera High School],[Charter],"1,858","$1,081,356.0",$582.0,83.06,83.98,94.13%,97.04%,95.58%
14,[Thomas High School],[Charter],"1,635","$1,043,130.0",$638.0,83.42,83.85,93.27%,97.31%,95.29%
9,[Pena High School],[Charter],962,"$585,858.0",$609.0,83.84,84.04,94.59%,95.95%,95.27%
4,[Griffin High School],[Charter],"1,468","$917,500.0",$625.0,83.35,83.82,93.39%,97.14%,95.26%
5,[Wilson High School],[Charter],"2,283","$1,319,574.0",$578.0,83.27,83.99,93.87%,96.54%,95.21%


In [265]:
# Sort and display the five worst-performing schools
school_level_df = school_level_df.sort_values("Overall Pass Rate", ascending=True)
school_level_df.head(5)

,Schools Names,Types,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Math Pass,% Reading Pass,Overall Pass Rate
School ID,,,,,,,,,,
11,[Rodriguez High School],[District],"3,999","$2,547,363.0",$637.0,76.84,80.74,66.37%,80.22%,73.29%
1,[Figueroa High School],[District],"2,949","$1,884,411.0",$639.0,76.71,81.16,65.99%,80.74%,73.37%
0,[Huang High School],[District],"2,917","$1,910,635.0",$655.0,76.63,81.18,65.68%,81.32%,73.50%
12,[Johnson High School],[District],"4,761","$3,094,650.0",$650.0,77.07,80.97,66.06%,81.22%,73.64%
3,[Hernandez High School],[District],"4,635","$3,022,020.0",$652.0,77.29,80.93,66.75%,80.86%,73.81%


In [267]:
# Reading Scores by Grade
# Create a table that lists the average reading score for students of each grade level (9th, 10th, 11th, 12th) at each school.
# Create a pandas series for each grade. Hint: use a conditional statement.

grouped_school_df2 = school_data_complete.groupby(['School ID', 'grade'])

#print(grouped_school_df2["reading_score"].sum().head(10))
#print(grouped_school_df2["Student ID"].count().head(10))

avg_read_school_grade = grouped_school_df2["reading_score"].sum()/grouped_school_df2["Student ID"].count()
#print(avg_read_school_grade)


school_names = school_data_complete.school_name.unique()

# The following codes first extract all 9th grade data out and then groupby school ID to calculate reading scores
avg_read_grade9 = school_data_complete.loc[school_data_complete['grade'] == "9th"]
avg_read_school_grade9 = avg_read_grade9.groupby(['School ID'])
reading_score9 = avg_read_school_grade9["reading_score"].sum()/avg_read_school_grade9["Student ID"].count()


avg_read_grade10 = school_data_complete.loc[school_data_complete['grade'] == "10th"]
avg_read_school_grade10 = avg_read_grade10.groupby(['School ID'])
reading_score10 = avg_read_school_grade10["reading_score"].sum()/avg_read_school_grade10["Student ID"].count()


avg_read_grade11 = school_data_complete.loc[school_data_complete['grade'] == "11th"]
avg_read_school_grade11 = avg_read_grade11.groupby(['School ID'])
reading_score11 = avg_read_school_grade11["reading_score"].sum()/avg_read_school_grade11["Student ID"].count()


avg_read_grade12 = school_data_complete.loc[school_data_complete['grade'] == "12th"]
avg_read_school_grade12 = avg_read_grade12.groupby(['School ID'])
reading_score12 = avg_read_school_grade12["reading_score"].sum()/avg_read_school_grade12["Student ID"].count()



reading_school_grade = pd.DataFrame({
    "Schools Names": school_names, 
    "9th": reading_score9,
    "10th": reading_score10,
    "11th": reading_score11,
    "12th": reading_score12
})
reading_school_grade = reading_school_grade.set_index('Schools Names')
reading_school_grade = reading_school_grade.sort_values("Schools Names", ascending=True)


reading_school_grade["9th"] = reading_school_grade["9th"].map("{:.2f}".format)
reading_school_grade["10th"] = reading_school_grade["10th"].map("{:.2f}".format)
reading_school_grade["11th"] = reading_school_grade["11th"].map("{:.2f}".format)
reading_school_grade["12th"] = reading_school_grade["12th"].map("{:.2f}".format)

print(reading_school_grade)

#Group each series by school

#Combine the series into a dataframe

#Optional: give the displayed data cleaner formatting

                         9th   10th   11th   12th
Schools Names                                    
Bailey High School     81.30  80.91  80.95  80.91
Cabrera High School    83.68  84.25  83.79  84.29
Figueroa High School   81.20  81.41  80.64  81.38
Ford High School       80.63  81.26  80.40  80.66
Griffin High School    83.37  83.71  84.29  84.01
Hernandez High School  80.87  80.66  81.40  80.86
Holden High School     83.68  83.32  83.82  84.70
Huang High School      81.29  81.51  81.42  80.31
Johnson High School    81.26  80.77  80.62  81.23
Pena High School       83.81  83.61  84.34  84.59
Rodriguez High School  80.99  80.63  80.86  80.38
Shelton High School    84.12  83.44  84.37  82.78
Thomas High School     83.73  84.25  83.59  83.83
Wilson High School     83.94  84.02  83.76  84.32
Wright High School     83.83  83.81  84.16  84.07


In [268]:
# Math Scores by Grade
grouped_school_df3 = school_data_complete.groupby(['School ID', 'grade'])

#print(grouped_school_df2["reading_score"].sum().head(10))
#print(grouped_school_df2["Student ID"].count().head(10))

avg_read_school_grade = grouped_school_df3["math_score"].sum()/grouped_school_df2["Student ID"].count()
# print(avg_read_school_grade)


school_names = school_data_complete.school_name.unique()

avg_math_grade9 = school_data_complete.loc[school_data_complete['grade'] == "9th"]
avg_math_school_grade9 = avg_math_grade9.groupby(['School ID'])
math_score9 = avg_math_school_grade9["math_score"].sum()/avg_math_school_grade9["Student ID"].count()


avg_math_grade10 = school_data_complete.loc[school_data_complete['grade'] == "10th"]
avg_math_school_grade10 = avg_read_grade10.groupby(['School ID'])
math_score10 = avg_read_school_grade10["math_score"].sum()/avg_read_school_grade10["Student ID"].count()


avg_math_grade11 = school_data_complete.loc[school_data_complete['grade'] == "11th"]
avg_math_school_grade11 = avg_read_grade11.groupby(['School ID'])
math_score11 = avg_read_school_grade11["math_score"].sum()/avg_read_school_grade11["Student ID"].count()


avg_math_grade12 = school_data_complete.loc[school_data_complete['grade'] == "12th"]
avg_math_school_grade12 = avg_read_grade12.groupby(['School ID'])
math_score12 = avg_read_school_grade12["math_score"].sum()/avg_read_school_grade12["Student ID"].count()



math_school_grade = pd.DataFrame({
    "Schools Names": school_names, 
    "9th": math_score9,
    "10th": math_score10,
    "11th": math_score11,
    "12th": math_score12
})
math_school_grade = math_school_grade.set_index('Schools Names')
math_school_grade = math_school_grade.sort_values("Schools Names", ascending=True)


math_school_grade["9th"] = math_school_grade["9th"].map("{:.2f}".format)
math_school_grade["10th"] = math_school_grade["10th"].map("{:.2f}".format)
math_school_grade["11th"] = math_school_grade["11th"].map("{:.2f}".format)
math_school_grade["12th"] = math_school_grade["12th"].map("{:.2f}".format)

print(math_school_grade)


                         9th   10th   11th   12th
Schools Names                                    
Bailey High School     77.08  77.00  77.52  76.49
Cabrera High School    83.09  83.15  82.77  83.28
Figueroa High School   76.40  76.54  76.88  77.15
Ford High School       77.36  77.67  76.92  76.18
Griffin High School    82.04  84.23  83.84  83.36
Hernandez High School  77.44  77.34  77.14  77.19
Holden High School     83.79  83.43  85.00  82.86
Huang High School      77.03  75.91  76.45  77.23
Johnson High School    77.19  76.69  77.49  76.86
Pena High School       83.63  83.37  84.33  84.12
Rodriguez High School  76.86  76.61  76.40  77.69
Shelton High School    83.42  82.92  83.38  83.78
Thomas High School     83.59  83.09  83.50  83.50
Wilson High School     83.09  83.72  83.20  83.04
Wright High School     83.26  84.01  83.84  83.64


In [269]:
#print(school_data_complete.head())

# School names
# school_names = grouped_school_df.school_name.unique()
# print(school_names)
# Total Students
# school_n_students = grouped_school_df["Student ID"].count()
# Total School Budget
# school_budgets = grouped_school_df.budget.unique()
# Per Student Budget
# avg_budget = school_budgets/school_n_students



# Create a temparory dataframe to hold the average spending per student by school
school_level_df_temp = pd.DataFrame({
    "school_name": school_names, 
    "Per Student Budget": avg_budget
})

# need to transform it to a float data type to do calcualtions
school_level_df_temp["Per Student Budget"] = school_level_df_temp["Per Student Budget"].astype('float')
# Merge the per student budget spending back to the main data
school_spending = pd.merge(school_data_complete, school_level_df_temp, how="left", on=["school_name", "school_name"])
# print(school_spending.head())

# Using the bins given from the homework
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# Assigning the spending categories
school_spending["Spending Ranges(Per Student)"] = pd.cut(school_spending["Per Student Budget"], spending_bins, labels=group_names)


grouped_school_spending = school_spending.groupby(['Spending Ranges(Per Student)'])
# grouped_school_spending.head()

# Number of student per spending categories
spending_n_students = grouped_school_spending["Student ID"].count()
#print(spending_n_students)

# Average Math Score
avg_math_spending = grouped_school_spending.math_score.sum()/spending_n_students
# print(avg_math_spending)
# Average Reading Score
avg_read_spending = grouped_school_spending.reading_score.sum()/spending_n_students
# print(avg_read_spending)
# % Passing Math
pass_math_spending = round(grouped_school_spending["Math Pass 70"].sum()/spending_n_students, 4)*100
# print(pass_math_spending)
# % Passing Reading
pass_read_spending = round(grouped_school_spending["Reading Pass 70"].sum()/spending_n_students, 4)*100
# print(pass_read_spending)
# Overall Passing Rate (Average of the above two)
avg_pass_spending = round((pass_math_spending + pass_read_spending)/2,2)
# print(avg_pass_spending)



Avg_spending_students = pd.DataFrame({
    #"Per Student Budget": avg_budget,
    "Average Math Score": avg_math_spending,
    "Average Reading Score": avg_read_spending,  
    "% Math Pass": pass_math_spending,    
    "% Reading Pass": pass_read_spending,
    "Overall Pass Rate": avg_pass_spending
})
Avg_spending_students


,Average Math Score,Average Reading Score,% Math Pass,% Reading Pass,Overall Pass Rate
Spending Ranges(Per Student),,,,,
<$585,83.363065,83.964039,93.70,96.69,95.20
$585-615,83.529196,83.838414,94.12,95.89,95.00
$615-645,78.061635,81.434088,71.40,83.61,77.50
$645-675,77.049297,81.005604,66.23,81.11,73.67


In [272]:
# Create a temparory dataframe to hold the school size
school_size_temp = pd.DataFrame({
    "school_name": school_names, 
    "Total students": school_n_students
})

school_size = pd.merge(school_data_complete, school_size_temp, how="left", on=["school_name", "school_name"])
school_size.head()

# Sample bins from the homework
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_size["School Size"] = pd.cut(school_size["size"], size_bins, labels=group_names)
school_size.sample(20)


grouped_school_size = school_size.groupby(['School Size'])
grouped_school_size.count()


# Number of student per size categories
n_students = grouped_school_size["School Size"].count()
#print(spending_n_students)

# Average Math Score
avg_math = round(grouped_school_size.math_score.sum()/n_students,2)
# print(avg_math_spending)
# Average Reading Score
avg_read = round(grouped_school_size.reading_score.sum()/n_students,2)
# print(avg_read_spending)
# % Passing Math
pass_math = round(grouped_school_size["Math Pass 70"].sum()/n_students, 4)*100
# print(pass_math_spending)
# % Passing Reading
pass_read = round(grouped_school_size["Reading Pass 70"].sum()/n_students, 4)*100
# print(pass_read_spending)
# Overall Passing Rate (Average of the above two)
avg_pass = round((pass_math + pass_read)/2,2)
# print(avg_pass_spending)



Avg_size_students = pd.DataFrame({
    #"Per Student Budget": avg_budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,  
    "% Math Pass": pass_math,    
    "% Reading Pass": pass_read,
    "Overall Pass Rate": avg_pass
})
Avg_size_students


,Average Math Score,Average Reading Score,% Math Pass,% Reading Pass,Overall Pass Rate
School Size,,,,,
Small (<1000),83.83,83.97,93.95,96.04,95.00
Medium (1000-2000),83.37,83.87,93.62,96.77,95.20
Large (2000-5000),77.48,81.20,68.65,82.13,75.39


In [273]:
# school_data_complete.sample(10)

In [276]:
grouped_school_type = school_data_complete.groupby(['type'])

# Number of student per school type categories
n_students = grouped_school_type["type"].count()
#print(spending_n_students)

# Average Math Score
avg_math = round(grouped_school_type.math_score.sum()/n_students,2)
# print(avg_math_spending)
# Average Reading Score
avg_read = round(grouped_school_type.reading_score.sum()/n_students,2)
# print(avg_read_spending)
# % Passing Math
pass_math = round(grouped_school_type["Math Pass 70"].sum()/n_students, 4)*100
# print(pass_math_spending)
# % Passing Reading
pass_read = round(grouped_school_type["Reading Pass 70"].sum()/n_students, 4)*100
# print(pass_read_spending)
# Overall Passing Rate (Average of the above two)
avg_pass = round((pass_math + pass_read)/2,2)
# print(avg_pass_spending)



Avg_type_students = pd.DataFrame({
    #"Per Student Budget": avg_budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,  
    "% Math Pass": pass_math,    
    "% Reading Pass": pass_read,
    "Overall Pass Rate": avg_pass
})
Avg_type_students






,Average Math Score,Average Reading Score,% Math Pass,% Reading Pass,Overall Pass Rate
type,,,,,
Charter,83.41,83.90,93.70,96.65,95.18
District,76.99,80.96,66.52,80.91,73.72
